In [1]:
import os
import json
import time
import glob
import numpy as np
import pandas as pd
import datetime
from json.decoder import JSONDecodeError
from pandas import json_normalize
from matplotlib.backends.backend_pdf import PdfPages
import re
from nltk.stem import WordNetLemmatizer

from pandas_datareader import data as pdr

import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [2]:
def preprocess_results(day, top_p, tmp_path, with_sector):
    
    try:
        os.chdir(tmp_path)
    except:
        print('No such directory found!')

    print(tmp_path)

    '''
        get stock prices and large change stock list
    '''
    large_change_stocks_all_original = {}
    current_prices_original = {}

    for p in top_p:
        current_path = os.path.join(tmp_path, 'threshold_{}'.format(p))

        with open(current_path + '/1_large_change_stocks.json') as f:
            large_change_stocks_all_original[p] = json.load(f)

        with open(current_path + '/2_current_prices.json') as f:
            current_prices_original[p] = json.load(f)

    print('iterations: {}'.format(len(current_prices_original[top_p[0]].keys())))

    if day < '2020-11-01':
        open_time = datetime.datetime(2020, int(day.split('-')[1]), \
                                      int(day.split('-')[2]), 21, 31).isoformat()
    else:
        open_time = datetime.datetime(2020, int(day.split('-')[1]), \
                                  int(day.split('-')[2]), 22, 31).isoformat()

    print('Open time: {}'.format(open_time))

    large_change_stocks_all = large_change_stocks_all_original.copy()
    current_prices = current_prices_original.copy()

    if len([k for k, v in large_change_stocks_all_original[top_p[0]].items() if v != []]) == 0:
        print('no large change stocks!')

    if len(large_change_stocks_all_original[top_p[0]]['1']) != 0 and large_change_stocks_all_original[top_p[0]]['1'][0][3] < open_time:
        large_change_stocks_all[top_p[0]]['1'] = []

    '''
        sector_etf_prices_all
    '''
    if with_sector == True:
        # read from disk
        with open('{}/threshold_{}/4_sector_etf_prices.json'.format(tmp_path, top_p[0]), 'r') as fp:
            sector_etf_prices_dict = json.load(fp)

        # convert dictionaries into dataframes
        sector_etf_prices_all = {
            key: pd.DataFrame(sector_etf_prices_dict[key])
            for key in sector_etf_prices_dict
        }
    else:
        sector_etf_prices_all = {}

    return current_prices, large_change_stocks_all, sector_etf_prices_all



In [3]:
def get_first_large_change_times(current_stock, large_times):
    first_large_change = {}
    for p in top_p:
        first_large_change[p] = {}

        n_len = len(large_times[p])
        for i in range(n_len):
            if current_stock == large_times[p][i][0]:
                first_large_change[p][current_stock] = (large_times[p][i][3], large_times[p][i][1], large_times[p][i][2])
                break
            else:
                continue
    return first_large_change


In [4]:
'''
   
   Get opening price as the first data point
        
'''
from pandas_datareader import data as pdr

def get_stock_openprice(unique_stocks, day):
    
    large_stock_openprice = {}
    large_stock_hist = {}

    for s in sorted(list(unique_stocks)):
        tmp_hist = pdr.get_data_yahoo(s, start = day, end = day)
        large_stock_hist[s] = tmp_hist
        tmp_hist = tmp_hist[~tmp_hist.index.duplicated(keep='first')]

        large_stock_openprice[s] = tmp_hist.loc[day]

    return large_stock_openprice

In [8]:
def plot_stock_prices(day, top_p, df_thre_at_q, relative_thre_at_q_lists, 
                     current_prices, large_change_stocks_all, 
                     sector_etf_prices_all,
                     pic_path, with_sector):

    if len([k for k, v in large_change_stocks_all[top_p[0]].items() if v != []]) == 0:
        print('No large change stock found!')
    else:

        '''
           stocks_list: 
           n_stocks: number of stocks
        '''

        stocks_list = [x[0] for x in current_prices[top_p[0]]['2']]
        n_stocks = len(stocks_list)

        print('Number of stocks in the list:', n_stocks)


        '''
            Get individual stock prices
                - individual_stock_price: 
                    - key: stock symbol
                    - value: key - timestamp, value - stock price
        '''
        individual_stock_price = {}
        for s in stocks_list:
        #     print(s)
            individual_stock_price[s] = {}

            for p in top_p:
                n_cnts = len(current_prices[p].keys())

                for k, v in current_prices[p].items():
                    for i in v:
                        if i[0] == s:
                            individual_stock_price[s][datetime.datetime.strptime(i[1], "%Y-%m-%dT%H:%M:%S.%f").isoformat()] = \
                                float(str(i[2]).replace(',',''))


        '''
            large_times:
                key: 95th
                value: (symbol, price, %change, time)
            large_stocks:
                key: 95th
                value: set of large change stocks
        '''
        large_times = {}
        large_stocks = {}

        for p in top_p:

            large_times[p] = []
            large_stocks[p] = []

            for k, v in large_change_stocks_all[p].items():
                if len(v) > 0:
                    large_stocks[p].extend([x[0] for x in v])
                    large_times[p].extend([(x[0], float(x[1].replace(',', '')), x[2], datetime.datetime.strptime(x[3], "%Y-%m-%dT%H:%M:%S.%f").isoformat()) for x in v])

            large_stocks[p] = set(large_stocks[p])

        print(large_stocks[top_p[0]], large_times[top_p[0]][0])


        '''
            first_large_change: 
                key: symbol
                value: dict of 
                    key: 95th
                    value: k=symbol, v=(time, price, %change)
        '''

        first_large_change = {}

        unique_stocks = set()
        for k,v in large_stocks.items():
            unique_stocks = unique_stocks.union(v)

        for s in sorted(list(unique_stocks)):
            first_large_change[s] = get_first_large_change_times(s, large_times)

        print(first_large_change)


        '''
            large_stock_price:
                key: symbol
                value: df-stock price for each iteration, index=time, value=price
        '''
        large_stock_price = {}
        for k, v in individual_stock_price.items():
            if k in unique_stocks:
                large_stock_price[k] = v


        for k, v in large_stock_price.items():
            large_stock_price[k] = pd.DataFrame(v.values(), index=v.keys(), columns=[k])
            large_stock_price[k].sort_index(inplace=True)

#         print(large_stock_price.keys())

        
        '''
            sector_stock_price:
                key: symbol
                value: df-stock price for each iteration, index=time, value=price

        '''
        if with_sector == True:
            sector_stock_price = {}

            for s in sector_list:
                sector_stock_price[s] = {}
                for k, v in sector_etf_prices_all.items():
                    tmp_datetime = v[v['Symbol'] == s].iloc[0, 1]
                    tmp_price = v[v['Symbol'] == s].iloc[0, 2]
                    sector_stock_price[s][tmp_datetime] = float(str(tmp_price).replace(',',''))

            for k, v in sector_stock_price.items():
                sector_stock_price[k] = pd.DataFrame(v.values(), index=v.keys(), columns=[k])
                sector_stock_price[k].sort_index(inplace=True)

#             print(sector_stock_price.keys())

        '''
            Stock price change plot:
                - red dot: first detect large price change
        '''

        large_stock_price_copy = large_stock_price.copy()


        if day < '2020-11-01':
            open_time = datetime.datetime(2020, int(day.split('-')[1]), int(day.split('-')[2]), 21, 31).isoformat()
        else:
            open_time = datetime.datetime(2020, int(day.split('-')[1]), int(day.split('-')[2]), 22, 31).isoformat()

        n_col = ['ro', 'bo', 'yo']


        if not os.path.exists(pic_path):
            os.mkdir(pic_path)

        with PdfPages('{}/all_stocks_price_plot_{}.pdf'.format(pic_path, day)) as pdf:
            for s in sorted(list(unique_stocks)):
                p_time = []
                for p in top_p:
                    if first_large_change[s][p]:
                        p_time.append((p, first_large_change[s][p][s]))
                print(s, p_time)

                if large_stock_price[s].index[0] > open_time:
                    ####
                    large_stock_openprice = get_stock_openprice(unique_stocks, day)

                    # adding open price in the stock price list 
                    large_stock_price_copy[s] = pd.concat([pd.DataFrame({s: large_stock_openprice[s].loc['Open']}, index=[open_time]), \
                              large_stock_price_copy[s]])

                large_stock_price_copy[s] = large_stock_price_copy[s].sort_index()
                
                # get length of stock prices
                p_len = len(large_stock_price_copy[s].index)

                insert_index = [(p_time[i][0], sum(p_time[i][1][0] > large_stock_price_copy[s].index)) for i in range(len(p_time))]
                print(insert_index)

                s_change = [(large_stock_price_copy[s].iloc[i, 0] - \
                            large_stock_price_copy[s].iloc[0, 0])/large_stock_price_copy[s].iloc[0, 0]*100 for i in range(large_stock_price_copy[s].shape[0])]

                if with_sector == True:
                    tmp_sector = stocks_combine_list_with_sector[stocks_combine_list_with_sector['Symbol'] == s]['Sector_ETF'].values[0]
                    print(tmp_sector)

                    # relative % change: 
                    sector_change = [(sector_stock_price[tmp_sector].iloc[i, 0] - \
                                     sector_stock_price[tmp_sector].iloc[0, 0])/sector_stock_price[tmp_sector].iloc[0, 0]*100 for i in range(sector_stock_price[tmp_sector].shape[0])]

                    relative_change = [round((s_change[i] - sector_c)/sector_c, 1) for i, sector_c in enumerate(sector_change)]


                if insert_index[0][1] > 1:
                    
                    if with_sector == True:
                        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5))

                        # subplot: 1
                        ax1.plot(range(p_len), large_stock_price_copy[s].values, 'o-')

                        dot = {}
                        for i in range(len(p_time)):
                            dot[i], = ax1.plot(insert_index[i][1]-1, p_time[i][1][1], n_col[i])

                        ax1.legend(list(dot.values()), ['Relative diff against {}: {}%'.format(tmp_sector, round(p_time[i][1][2], 2)) for p in p_time], loc = 4)
                        ax1.set_xlabel('Iteration')
                        ax1.set_ylabel('Price')
                        ax1.set_title('{}: {} ({})'.format(day, stocks_combine_list_with_sector[stocks_combine_list_with_sector['Symbol'] == s].iloc[0]['Name'], s))
                        txt = "Stock {}: {} relative difference (2.5th and 97.5th: {}) and 95th-threshold: {}% \n at time {} with price ${}".format(s, 
                                             round(p_time[0][1][2], 2), [round(x, 2) for x in relative_thre_at_q_lists[stocks_combine_list_with_sector[stocks_combine_list_with_sector['Symbol'] == s].index[0]]],
                                             round(df_thre_at_q[df_thre_at_q['symbols']==s]['threshold_at_{}'.format(top_p[0])].values[0], 2),
                                             p_time[0][1][0], p_time[0][1][1])
                        ax1.text(0.05, 0.95, txt, transform=fig.transFigure, size = 8)
                        
                        for i, txt in enumerate(s_change):
                            ax1.annotate('{}%'.format(round(txt,1)), (i+0.4, large_stock_price_copy[s].values[i]))


                        # subplot: 2
                        ax2.plot(range(len(sector_stock_price[tmp_sector].index)), 
                             sector_stock_price[tmp_sector].values, 'o-', label = tmp_sector)

                        dot = {}
                        for i in range(len(p_time)):
                            dot[i], = ax2.plot(insert_index[i][1]-1, sector_stock_price[tmp_sector].iloc[insert_index[i][1]-1, 0], 
                                               n_col[i], label = 'large_change')


                        for i, txt in enumerate(relative_change):
                            ax2.annotate(txt, (i+0.4, sector_stock_price[tmp_sector].values[i]))


                        ax2.set_xlabel('Iteration')
                        ax2.set_ylabel('Price')
                        ax2.legend()
                        ax2.set_title('{}: {} ({})'.format(day, stocks_combine_list_with_sector[stocks_combine_list_with_sector['Sector_ETF'] == tmp_sector].iloc[0]['Sector'], tmp_sector))

                        pdf.savefig(fig)
                        plt.close()
                    else:

                        fig = plt.figure(figsize=(7, 5))
                        plt.plot(range(p_len), large_stock_price_copy[s].values, 'o-')

                        dot = {}
                        for i in range(len(p_time)):
                            dot[i], = plt.plot(insert_index[i][1]-1, p_time[i][1][1], n_col[i])

                        plt.legend(list(dot.values()), ['Percentage Change: {}%'.format(p_time[i][1][2]) for p in p_time], loc = 4)
                        plt.xlabel('Iteration')
                        plt.ylabel('Price')
                        plt.title('{}: {} ({})'.format(day, stocks_combine_list_with_sector[stocks_combine_list_with_sector['Symbol'] == s].iloc[0]['Name'], s))
                        txt = "Stock {}: {}% change (threshold {}%) \n at time {} with price ${}".format(s, 
                                             p_time[0][1][2], round(df_thre_at_q[df_thre_at_q['symbols']==s]['threshold_at_{}'.format(top_p[0])].values[0], 2), 
                                             p_time[0][1][0], p_time[0][1][1])
                        plt.text(0.05, 0.95, txt, transform=fig.transFigure, size = 8)

                        for i, txt in enumerate(s_change):
                            plt.annotate('{}%'.format(round(txt,1)), (i+0.4, large_stock_price_copy[s].values[i]))

                        pdf.savefig(fig)
                        plt.close()


    
        return unique_stocks, first_large_change

In [6]:
'''
    dates_filename_list: 
    top_p: 
        95th percentile as the threshold to detect a stock with large change
    sub_industry: 
        focus on mid-cap and large-cap companies

'''
parent_dir = '/Users/lin/Downloads/Q3_Web_scraping/'

sub_industry = 'mid_large_cap_stocks'

top_p = [95]

dates_filename_list = sorted([filename for filename in os.listdir('{}mid_large_cap_stocks/'.format(parent_dir)) 
                              if filename.startswith('2020')])
print(dates_filename_list)


df_thre_at_q = pd.read_csv('{}df_thre_at_q{}.csv'.format(parent_dir,\
                                                               top_p[0]), index_col=0)

# load
with open("{}/relative_thre_at_q_lists.json".format(os.path.join(parent_dir, 'mid_large_cap_stocks'))) as f:
    relative_thre_at_q_lists = json.loads(f.read())

['2020-10-02', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-11-03', '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-09', '2020-11-10']


In [7]:
stocks_combine_list_with_sector = pd.read_csv('{}/{}/stocks_combine_list_with_sector.csv'.format(parent_dir, sub_industry), 
                                             index_col=0)
sector_list = sorted(list(stocks_combine_list_with_sector['Sector_ETF'].unique()))
print(sector_list)

stocks_combine_list_with_sector.head()

['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']


,Symbol,Size,Market_Cap,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location,Sector_ETF
0,AAPL,large-cap,1.876243e+12,Apple Inc,Common Stock,315.01,Information Technology,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",NASDAQ,United States of America,XLK
1,AB,mid-cap,2.589199e+09,AllianceBernstein Holding LP,Unit Trust Fund,22.51,Financials,Capital Markets,Asset Management & Custody Banks,NYSE,United States of America,XLF
2,ABT,large-cap,1.786819e+11,Abbott Laboratories,Common Stock,96.07,Health Care,Health Care Equipment & Supplies,Health Care Equipment,NYSE,United States of America,XLV
3,ACC,mid-cap,4.614801e+09,American Campus Communities Inc,Common Stock (REIT),30.59,Real Estate,Equity Real Estate Investment Trusts (REITs),Residential REITs,NYSE,United States of America,XLRE
4,ADBE,large-cap,2.243249e+11,Adobe Inc,Common Stock,371.42,Information Technology,Software,Application Software,NASDAQ,United States of America,XLK


In [12]:
dates_filename_list[-1:]

['2020-11-10']

In [13]:

'''
    Stock price movement visualization
        1. 2020/10/02 - 2020/11/03: %change compared with 95th percentile threshold obtained from past 1-year data
        2. 2020/11/04 - present: get relative difference of the %change between individual stock and corresponding sector ETF
                                then, compared the relative difference with the 2.5th and 97.5th percentile thresholds
                                obtained from past 1-year data

'''

# current_prices_d = {}
# large_change_stocks_all_d ={}
# sector_etf_prices_all_d = {}

# unique_stocks_d = {}
# first_large_change_d = {}

with_sector = False
# dates_filename_list

for day in dates_filename_list[-1:]:
    print(day)

    tmp_path = os.path.join(os.path.join(parent_dir, sub_industry),  day)

    if day > '2020-11-03':
        with_sector = True
    else:
        with_sector = False
        
    current_prices_d[day], \
    large_change_stocks_all_d[day], \
    sector_etf_prices_all_d[day] = preprocess_results(day, top_p, tmp_path, with_sector)
    
    if len([k for k, v in large_change_stocks_all_d[day][top_p[0]].items() if v != []]) == 0:
        print('No large change stock found!')
        unique_stocks_d[day] = set()
        first_large_change_d[day] = dict()
    else:
        unique_stocks_d[day], \
        first_large_change_d[day] = plot_stock_prices(day, top_p, df_thre_at_q, relative_thre_at_q_lists, 
                                                     current_prices_d[day], large_change_stocks_all_d[day],
                                                     sector_etf_prices_all_d[day], 
                                                     pic_path=os.path.join(parent_dir, 'stock_price_figures'), 
                                                     with_sector=with_sector)

    print('\n')

2020-11-10
/Users/lin/Downloads/Q3_Web_scraping/mid_large_cap_stocks/2020-11-10
iterations: 26
Open time: 2020-11-10T22:31:00
Number of stocks in the list: 167
{'IDXX', 'QDEL', 'TRUP', 'SLM', 'MCD', 'ORLY', 'ALLY', 'CCI', 'FVRR', 'HIW', 'WELL', 'IBKC', 'UA', 'EQR', 'EPR', 'PPD', 'ACC', 'TWST', 'WAT', 'RL', 'TM', 'CHDN', 'PHM', 'ISRG', 'TPH', 'MS', 'CMA', 'HTHT', 'MPW', 'REGN', 'PINS', 'LLY', 'STAY', 'ALL', 'CPB', 'JNJ', 'FLO', 'TROW', 'UMPQ', 'AMT', 'PLD', 'KMX', 'QGEN', 'PRU', 'HBAN', 'SF', 'ARE', 'SPOT', 'TFX', 'CR', 'NVCR', 'BYD', 'AXS', 'CVS', 'SRPT', 'AMED', 'BNS', 'HSBC', 'ALGN', 'O', 'F', 'TWTR', 'ADC', 'AMP', 'AB', 'DRE', 'HOMB', 'WBK'} ('AMP', 180.97, 14.428571428571427, '2020-11-10T22:39:35.972646')
{'AB': {95: {'AB': ('2020-11-11T01:38:29.064829', 31.78, 12.727272727272727)}}, 'ACC': {95: {'ACC': ('2020-11-10T22:55:39.186822', 40.76, 15.818181818181818)}}, 'ADC': {95: {'ADC': ('2020-11-10T22:55:39.186822', 64.98, 20.454545454545453)}}, 'ALGN': {95: {'ALGN': ('2020-11-10T23:1

/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars
/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


ADC [(95, ('2020-11-10T22:55:39.186822', 64.98, 20.454545454545453))]
[(95, 3)]
XLRE
ALGN [(95, ('2020-11-10T23:10:46.753503', 466.78, 213.00000000000003))]
[(95, 4)]
XLV


/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars
/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


ALL [(95, ('2020-11-11T00:23:50.066322', 93.48, 13.666666666666668))]
[(95, 9)]
XLF


...

/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars
/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


UMPQ [(95, ('2020-11-11T01:53:14.309813', 14.32, 16.05263157894737))]
[(95, 15)]
XLF
WAT [(95, ('2020-11-10T23:10:46.753503', 227.12, 143.0))]
[(95, 4)]
XLV


/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars
/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


WBK [(95, ('2020-11-10T22:39:35.972646', 13.65, 10.428571428571427))]
[(95, 2)]
XLF
WELL [(95, ('2020-11-10T22:55:39.186822', 68.38, 22.454545454545457))]
[(95, 3)]
XLRE


/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars
/Users/lin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


## News and UGC data

In [8]:

'''
   
   Get opening price as the first data point
        
'''


def get_large_stock_df(day, unique_stocks_d, first_large_change_d):
    

    large_stock_openprice = {}
    large_stock_hist = {}

    for s in sorted(list(unique_stocks_d[day])):
        tmp_hist = pdr.get_data_yahoo(s, start = day, end = day)
        large_stock_hist[s] = tmp_hist
        tmp_hist = tmp_hist[~tmp_hist.index.duplicated(keep='first')]

        large_stock_openprice[s] = tmp_hist.loc[day]


    large_stock_df = pd.DataFrame(0, index=sorted(list(unique_stocks_d[day])), 
                                  columns=large_stock_hist[s].columns)
    large_stock_df['Thre_at_{}'.format(top_p[0])] = 0
    large_stock_df['First_Percent_Change'] = 0
    large_stock_df['First_Stock_Price'] = 0

    for s in sorted(list(unique_stocks_d[day])):
        large_stock_df.loc[s] = large_stock_hist[s].loc[day]
        large_stock_df.loc[s, 'Thre_at_{}'.format(top_p[0])] = df_thre_at_q[df_thre_at_q['symbols']==s]['threshold_at_{}'.format(top_p[0])].values
        large_stock_df.loc[s, 'First_Percent_Change'] = first_large_change_d[day][s][top_p[0]][s][2]
        large_stock_df.loc[s, 'First_Stock_Price'] = first_large_change_d[day][s][top_p[0]][s][1]

    large_stock_df = large_stock_df.assign(Total_Percent_Change = lambda x: 
                                                           round((x['Close'] - x['Open'])/x['Open']*100 ,2))

    print(large_stock_df.shape)

    large_stock_df.reset_index(inplace=True)
    large_stock_df.rename(columns={'index': 'Symbol'}, inplace=True)

    large_stock_df = pd.merge(large_stock_df, stocks_midlargeCap_webscrape_df,
                        how = 'left', on = 'Symbol')

    return large_stock_df

In [9]:
'''
    Get News and UGC dataframe for each day
'''


def get_news_ugc_data(day, tmp_path, unique_stocks_d):

    res_news_df = {}
    res_ugc_df = {}

    for s in sorted(list(unique_stocks_d[day])):

        news_filename_list = glob.glob(os.path.join(tmp_path, '{}/News_*.json'.format(s)))
        ugc_filename_list = glob.glob(os.path.join(tmp_path, '{}/UGC_*.json'.format(s)))

        res_news_dict = {}
        res_ugc_dict = {}

        for news_file in news_filename_list:

            with open(news_file) as f:
                try:
                    res_news_dict[news_file.split('{}/threshold_{}/{}/'.format(day, top_p[0], s))[1]] = json.load(f)
                except JSONDecodeError:
                    pass
            f.close()

        res_news_df[s] = pd.DataFrame({'published': [item['published'] for k, v in res_news_dict.items() for item in v],
                                    'title':[item['title'] for k, v in res_news_dict.items() for item in v]})
        # drop duplicates by title:
        res_news_df[s] = res_news_df[s].drop_duplicates('title').reset_index(drop = True)

        # drop news that do not contain stock key-words in title:
        res_news_df[s] = res_news_df[s][res_news_df[s]['title'].map(lambda x: \
                 any(word in x.lower() for word in stocks_key_words_dict[s]))].reset_index(drop=True)

        print('{}: News dataset shape: {}'.format(s, res_news_df[s].shape))

        for ugc_file in ugc_filename_list:

            with open(ugc_file) as f:
                try:
                    res_ugc_dict[ugc_file.split('{}/threshold_{}/{}/'.format(day, top_p[0], s))[1]] = json.load(f)
                except JSONDecodeError:
                    pass
            f.close()

        res_ugc_df[s] = []
        for k, v in res_ugc_dict.items():
            tmp_df = pd.DataFrame.from_dict(json_normalize(v['messages']), orient='columns')
            res_ugc_df[s].append(tmp_df)

        res_ugc_df[s] = pd.concat(res_ugc_df[s])
        # drop duplicates by message id:
        res_ugc_df[s] = res_ugc_df[s].drop_duplicates('id').reset_index(drop = True)

        print('{}: UGC dataset shape: {}'.format(s, res_ugc_df[s].shape))
    
    return res_news_df, res_ugc_df

In [10]:
stocks_midlargeCap_webscrape_df = pd.read_csv('{}/df_stocks_midlargeCap_webscrape.csv'.format(parent_dir), 
                                              index_col=0)

print(stocks_midlargeCap_webscrape_df.shape)

(167, 11)


In [11]:

'''

    Loading News and UGC data

        stocks_key_words_dict
        large_stock_df_dict
    
    Results: News and UGC
        res_news_df_dict
        res_ugc_df_dict

    
'''
# stocks_key_words_dict
stocks_key_words_dict = {}
del_words = ['Ltd', 'Inc', 'Corp', 'Holdings', 'LLC', 'Corporation', 'PLC',
             'HOLDERs', 'Co', 'Limited', 'Company',
             'Providers', 'group', 'com', 'Brands']
del_words = [w.lower() for w in del_words]

w_lemmatizer = WordNetLemmatizer()

# large_stock_df_dict
large_stock_df_dict = {}

# results: News and UGC
res_news_df_dict = {}
res_ugc_df_dict = {}

for day in dates_filename_list:
    print(day)
    
    if unique_stocks_d[day] == set():
        print('No large change stock found in day {}'.format(day))
    else:
        print('Found large change stocks:', unique_stocks_d[day])
        
        large_stock_df = get_large_stock_df(day, unique_stocks_d, first_large_change_d)
        large_stock_df_dict[day] = large_stock_df

        for s in sorted(list(unique_stocks_d[day])):
            if s not in stocks_key_words_dict:
                tmp_words = []
                tmp_words.append(s.lower())

                tmp_names = [w_lemmatizer.lemmatize(w.lower()) for w in re.findall(r'\w+', large_stock_df[large_stock_df['Symbol'] == s]['Name'].values[0]) \
                             if w.lower() not in del_words]
                tmp_words.append(' '.join(tmp_names))

                tmp_words = list(set(tmp_words))
                stocks_key_words_dict[s] = tmp_words

        tmp_path = os.path.join(os.path.join(os.path.join(parent_dir, 'mid_large_cap_stocks'), 
                 day), 'threshold_{}/'.format(top_p[0]))

        res_news_df_dict[day], \
        res_ugc_df_dict[day] = get_news_ugc_data(day, tmp_path, unique_stocks_d)

        print('\n')

2020-10-02
Found large change stocks: {'HSBC', 'LSCC'}
(2, 10)
HSBC: News dataset shape: (2, 2)
HSBC: UGC dataset shape: (30, 42)
LSCC: News dataset shape: (0, 2)
LSCC: UGC dataset shape: (30, 34)


2020-10-05
Found large change stocks: {'EPR', 'LSCC', 'MYOK', 'REGN', 'AGIO', 'AQN', 'ZION', 'ROK', 'PBR', 'WIT', 'MRVL', 'STLD', 'UMC'}
(13, 10)
AGIO: News dataset shape: (3, 2)
AGIO: UGC dataset shape: (30, 40)
AQN: News dataset shape: (0, 2)
AQN: UGC dataset shape: (30, 40)
EPR: News dataset shape: (9, 2)
EPR: UGC dataset shape: (61, 41)
LSCC: News dataset shape: (0, 2)
LSCC: UGC dataset shape: (30, 34)
MRVL: News dataset shape: (5, 2)
MRVL: UGC dataset shape: (36, 41)
MYOK: News dataset shape: (36, 2)
MYOK: UGC dataset shape: (114, 42)
PBR: News dataset shape: (0, 0)
PBR: UGC dataset shape: (30, 43)
REGN: News dataset shape: (14, 2)
REGN: UGC dataset shape: (428, 43)
ROK: News dataset shape: (12, 2)
ROK: UGC dataset shape: (32, 34)
STLD: News dataset shape: (0, 2)
STLD: UGC dataset shap

#### Word frequency

In [12]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
from nltk.stem import WordNetLemmatizer

nlp = spacy.load('en_core_web_sm')

del_words.extend(['$', '\n', '\n\n', '|'])

In [13]:
day = dates_filename_list[0]
s = sorted(list(unique_stocks_d[day]))[0]
print(day, s)

stocks_key_words_dict[s]

2020-10-02 HSBC


['hsbc']

In [14]:
'''
    Word frequency:
        News
    
        UGC
    
'''
# news
tmp_news_title = res_news_df_dict[day][s]['title'].apply(lambda x: nlp(x))

news_words = [token.text for sent in tmp_news_title for token in sent if not token.is_stop and not token.is_punct and not token.is_space]
news_words = [w_lemmatizer.lemmatize(w.lower()) for w in news_words if w.lower() not in del_words]

news_word_freq = Counter(news_words)

# UGC
tmp_ugc_data = res_ugc_df_dict[day][s]['body'].apply(lambda x: nlp(x))

ugc_words = [token.text for sent in tmp_ugc_data for token in sent if not token.is_stop and not token.is_punct and not token.is_space]
ugc_words = [w_lemmatizer.lemmatize(w.lower()) for w in ugc_words if w.lower() not in del_words]

ugc_word_freq = Counter(ugc_words)

In [15]:
news_word_freq.most_common(20)

[('hsbc', 3),
 ('bank', 3),
 ('market', 2),
 ('nyse', 1),
 ('currently', 1),
 ('99.45', 1),
 ('52-week', 1),
 ('high', 1),
 ('upside', 1),
 ('potential', 1),
 ('surprise', 1),
 ('marketing', 1),
 ('sentinel', 1),
 ('oman', 1),
 ('card', 1),
 ('payment', 1),
 ('report-', 1),
 ('growth', 1),
 ('opportunity', 1),
 ('company', 1)]

In [16]:
ugc_word_freq.most_common(20)

[('hsbc', 37),
 ('pm', 11),
 ('adr', 7),
 ('short', 6),
 ('stock', 6),
 ('form', 4),
 ('6-k', 4),
 ('filed', 4),
 ('sentiment', 4),
 ('close', 3),
 ('li', 3),
 ('15', 3),
 ('jpm', 3),
 ('negative', 3),
 ('social', 3),
 ('medium', 3),
 ('https://socialsentiment.io/stocks/symbol/hsbc/', 3),
 ('week', 3),
 ('+', 3),
 ('soon', 2)]

### PoS tagging

In [17]:
news_words_noun_verb_adj = [(token.text, token.pos_) for sent in tmp_news_title for token in sent \
 if not token.is_stop and not token.is_punct and not token.is_space \
 and token.pos_ in ('VERB', 'NOUN', 'ADJ')]
news_words_noun_verb_adj = [(w_lemmatizer.lemmatize(w.lower()), t) for w, t in news_words_noun_verb_adj \
                           if w.lower() not in del_words]


In [18]:
Counter(news_words_noun_verb_adj).most_common(20)

[(('high', 'ADJ'), 1),
 (('surprise', 'VERB'), 1),
 (('marketing', 'NOUN'), 1),
 (('sentinel', 'NOUN'), 1),
 (('market', 'NOUN'), 1)]